In [4]:
from pathlib import Path

import pandas as pd 

ModuleNotFoundError: No module named 'pandas'

In [3]:
activities = {"D01": "Walking", "D02":"Jogging", "D07": "Stumble", "D08": "Jump"}

In [ ]:
falls = Path("WEDA-FALL/dataset/fall_timestamps.csv")


In [2]:
dataset_path = Path("WEDA-FALL/dataset/50Hz/D*")
dataset_path

WindowsPath('WEDA-FALL/dataset/50Hz/D*')